Model Requirements:

1. **Decision Variables**:

- Weight matrix, according which producers distribute energy

1. **Constraints**:

- A consumer can only receive energy from producers they are connected to (preference > 0).
- A consumer can only receive energy up to their demand.
- A producer can only distribute up to their available supply.
- Respect the fixed limit of 5 connections per consumer.

1. **Objective Function**:

- Minimize the total unredistributed energy across all timestamps:
- Energy left with producers due to no eligible or matching demand.
- Unmet demand from consumers.


### Output:

- Total unredistributed energy over the full month.
- A breakdown of which consumers didn’t receive their full demand and which producers had unused energy. All of this output to a file for later analysis.

### Details:

Producers have weight vectors for distributing energy. Subscribers have preference vectors for up to 5 producers these preferences are given by users.The function evaluates the energy distribution for each timestamp, calculates unredistributed energy, and tracks remaining demand for each subscriber.



## Model formulation

### Input data

- **$T$**
  - set of timestamps $t$
- **$R$**
  - Round of allocation from 1 to 5
- **$e$**
  - Matrix with excess energy for all the producers at time $t$
- **$d$**
  - Matrix with deficit energy for all the consumers at time $t$
- **$X$**
  - Matrix with binary input, linking weights.
  - 1 on place $i$,$j$ means that flow is possible from $i$ to $j$

### Variables

- **$W$**
  - Matrix of weights (producers, consumers)
  - What proportion of producer energy goes to which consumer
- **$S$**
  - Data-frame with time excess energy for each time $t$
  - Also rounds are added, for each time $t$
- **$D$**
  - Data-frame with time deficit energy for each time $t$
  - Also rounds are added, for each time $t$
- **$F$**
  - Energy flow from in time $t$, from $i$  to $j$ in round $R$

### Constraints

- $\sum_{j \in C} W^{i, j} = 1$, $\forall i \in P$
  - No producer can give more than 100% of its energy
- $W^{i,j} \leq X^{i,j}$ $\forall i \in P$, $\forall j \in C$
  - Eliminating non-zero weight where is no link
- $S^{i}_{t, 1} = e^{i}_{t}$, $\forall t \in T$ and $\forall i \in P$
  - Initial value of supply of energy
- $D^{j}_{t,1} = d^{t}_{j}$, $\forall t \in T$ and $\forall j \in C$
  - Initial value of demand of energy
- $F^{i,j}_{t,r} \leq W^{i, j} * S^{i}_{t, r-1} * X^{i, j}$, $\forall i \in P$, $\forall j \in C$, 
  - Flow cannot exceed the supply
- $S^{i}_{t, r} = S^{i}_{t, r-1} - F^{i, j}_{t, r-1}$, $\forall i \in P~t \in T,~r \in \{2,..,5\}$
  - Update of supplied energy
- $D^{j}_{t, r} = D^{j}_{t, r-1} - F^{i, j}_{t, r-1}$, $\forall j \in C ~t \in T,~r \in \{2,..,5\}$
  - Update of demanded energy

### Objective

- $Minimize~\sum_{i \in P, ~t \in T} S^{i}_{t,5} + \sum_{j \in C, ~t \in T,} D^{j}_{t,5}$

In [68]:
%load_ext autoreload
%autoreload 2

In [69]:
import numpy as np
import pandas as pd
import os
import utils as ut
from weights_optimization_model import optimize_weights

In [ ]:
data_dir = '/home/miro/Bachelor/BT/data/outputs/'

excess_monthly, deficit_monthly = ut.load_excess_deficit(data_dir)

month = 4 # May
start = 0
end   = 250

excess_df, deficit_df = ut.prepare_data(excess_monthly, deficit_monthly, month, start, end)
producers = excess_df.columns
consumers = deficit_df.columns
preference_df = ut.generate_preferences(producers, consumers)
preference_df = preference_df.loc[producers,consumers]

In [108]:
consumers = [ f'{i}' for i in range(7)]
producers = [f'{j}' for j in range(8)]

p = ut.generate_preferences(producers, consumers)

preferences = {}

for consumer in consumers:
    preferred_producers = np.random.choice([p for p in producers if p != consumer], size=5, replace=False)
    preference_values = np.random.permutation([i for i in range(1, 5+1)])
    preferences[consumer] = dict(zip(preferred_producers, preference_values))

preferences = pd.DataFrame(preferences).fillna(0)

preferences.T.loc[consumers, producers]

,0,1,2,3,4,5,6,7
0,0.0,5.0,3.0,0.0,2.0,1.0,4.0,0.0
1,4.0,0.0,5.0,0.0,1.0,0.0,3.0,2.0
2,0.0,0.0,0.0,4.0,5.0,3.0,1.0,2.0
3,1.0,0.0,4.0,0.0,0.0,2.0,3.0,5.0
4,0.0,3.0,2.0,4.0,0.0,5.0,0.0,1.0
5,4.0,2.0,3.0,5.0,0.0,0.0,0.0,1.0
6,0.0,4.0,2.0,0.0,1.0,3.0,0.0,5.0


In [70]:
opt_result = optimize_weights(excess_df, deficit_df, preference_df)
weights, unmet_demand, unused_supply = opt_result
fitness_score = ut.fitness(weights, preference_df, excess_df, deficit_df)

print(f'Objective function: {unmet_demand + unused_supply}')
print(f'fitness: {fitness_score}')

weights

Objective function: 18.430646528880235
fitness: unsatisfied_demand     1.979520
available_energy      16.578125
Total                 18.557645
dtype: float64


,zs_preislerova,zs_komenskeho,ms_preislerova,ms_pod_homolkou,ms_vrchlickeho,ms_drasarova,ms_na_machovne,zimni_stad,parkovaci_dum,radnice,ms_tovarni,dum_pro_duchodce,plavecky_areal,pristavba_preislerova
zs_preislerova,0.0,0.0,0.00000,0.000000,0.500,0.000000,0.222222,0.0,0.5000,0.0,0.0,1.0,0.0,0.000000
zs_komenskeho,0.0,0.0,0.00000,0.456675,0.000,0.000000,0.000000,0.5,0.0000,1.0,0.0,0.0,0.0,0.000000
ms_preislerova,0.0,0.0,0.00000,0.000000,0.000,0.000000,0.000000,0.0,0.0000,0.0,1.0,0.0,0.0,0.000000
ms_pod_homolkou,0.0,0.0,0.00000,0.000000,0.000,0.000000,0.000000,0.0,0.0000,0.0,0.0,0.0,0.0,0.082699
ms_vrchlickeho,0.0,0.0,0.00000,0.000000,0.000,0.000000,0.000000,0.0,0.0000,0.0,0.0,0.0,0.0,0.000000
ms_drasarova,0.0,0.0,0.00000,0.000000,0.000,0.000000,0.000000,0.0,0.0000,0.0,0.0,0.0,0.0,0.100000
ms_na_machovne,0.0,0.0,0.00000,0.000000,0.000,0.000000,0.000000,0.0,0.0000,0.0,0.0,0.0,0.0,0.000000
zimni_stad,0.0,0.0,0.00000,0.000000,0.000,0.000000,0.000000,0.0,0.0000,0.0,0.0,0.0,1.0,0.000000
parkovaci_dum,0.0,1.0,0.00000,0.500000,0.000,0.000000,0.000000,0.0,0.0000,0.0,0.0,0.0,0.0,0.509609
radnice,0.0,0.0,0.00000,0.000000,0.000,0.443200,0.000000,0.0,0.0000,0.0,0.0,0.0,0.0,0.307692


### Testing the model on fabricated data

What interests about the model?

- Is the process the same as a fitness function?
  - How to test this?
- what are the edge cases and test them.
  - 0 preference
  - equal preferences

In [43]:
time_index = pd.date_range("2025-01-01", periods=1, freq="15T")
excess_df = pd.DataFrame(
    [[100, 25]],# [40, 70], [30, 80]],
    index=time_index,
    columns=["P0", "P1"]
)
deficit_df = pd.DataFrame(
    [[15, 50, 5]], #[60, 20, 40], [30, 60, 20]],
    index=time_index,
    columns=["C0", "C1", "C2"]
)
preference_df = pd.DataFrame(
    [[5, 3, 4],
    [0, 0, 1]],
    index=["P0", "P1"],
    columns=["C0", "C1", "C2"]
)

opt_weights, unmet, unused = optimize_weights(
    excess_df, deficit_df, preference_df, num_rounds=1, write=True
)
model_obj = unmet + unused

weight_df = pd.DataFrame.from_dict(opt_weights, orient="index")

producer_energy_data = pd.concat([
    pd.Series(excess_df.index.astype(str), name="timestamp"),
    excess_df.reset_index(drop=True)
], axis=1)

subscriber_demand_data = pd.concat([
    pd.Series(deficit_df.index.astype(str), name="timestamp"),
    deficit_df.reset_index(drop=True)
], axis=1)

fitness_score = ut.fitness(
    weight_df, preference_df,
    producer_energy_data,
    subscriber_demand_data,
    rounds=1
)

print(f"Model objective (unmet+unused): {model_obj:.6f}")
print(f"Fitness function score       : {fitness_score:.6f}")


Model objective (unmet+unused): 55.000000
Fitness function score       : 55.000000


/tmp/ipykernel_115086/2681057877.py:1: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  time_index = pd.date_range("2025-01-01", periods=1, freq="15T")
/home/miro/Bachelor/BT/Models/utils.py:146: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if available_energy[i] <= 0:


In [87]:
w = pd.read_csv('/home/miro/Bachelor/BT/Models/outputs/weights_1.csv')
w.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
Unnamed: 0,zs_preislerova,zs_komenskeho,ms_preislerova,ms_pod_homolkou,ms_vrchlickeho,ms_drasarova,ms_na_machovne,zimni_stad,parkovaci_dum,radnice,ms_tovarni,dum_pro_duchodce,plavecky_areal,pristavba_preislerova
zs_preislerova,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
zs_komenskeho,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
ms_preislerova,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
ms_pod_homolkou,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
ms_vrchlickeho,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
ms_drasarova,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
ms_na_machovne,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
zimni_stad,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
parkovaci_dum,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [80]:
p = pd.read_csv('/home/miro/Bachelor/BT/Models/outputs/preferences.csv')
p.set_index("Unnamed: 0", inplace=True)

In [81]:
pref = p.values
links = (pref > 0).astype(int)
links

array([[0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0],
       [0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1],
       [0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0],
       [1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1],
       [1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1],
       [0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1],
       [0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0],
       [0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0],
       [1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1],
       [0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1],
       [0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0]])

In [45]:
0.0067677525 - (0.0035813247041052 + 0.0034199619717925 - 0.000233535)

8.241023002897307e-10